In [34]:
import pandas as pd
import seaborn
from geopy.distance import geodesic
import numpy as np

In [5]:
stages = pd.read_csv('stages_TDF.csv')
riders = pd.read_csv('riders')
cities = pd.read_csv('cities')

In [7]:
stages['Date'] = pd.to_datetime(stages['Date'])
stages['Year'] = stages['Date'].map(lambda x: x.year) # add year column
stages['rider_name'] = stages['Winner'].str.replace(' ', '_')

def changeStageNames(x):
    
    plainList = ['Plain stage', 'Flat Stage','Flat cobblestone stage','Plain stage with cobblestones','Intermediate stage', 'Team time trial', 'Half Stage']
    mountainList = ['Stage with mountain(s)','High mountain stage','Medium mountain stage','Stage with mountain','Mountain Stage', 'Hilly Stage', 'Transition stage']
    timeList = ['Mountain time trial', 'Individual time trial']
    
    if x in plainList:
        return 'Flat stage'
    if x in mountainList:
        return 'Mountain stage'
    if x in timeList:
        return 'Time trial'
        return x

stages['Gen_type'] = stages['Type'].apply(changeStageNames)

In [8]:
stages.head(5)

,Stage,Date,Distance,Origin,Destination,Type,Winner,Winner_Country,Year,rider_name,Gen_type
0,1,2017-07-01,14.0,Düsseldorf,Düsseldorf,Individual time trial,Geraint Thomas,GBR,2017,Geraint_Thomas,Time trial
1,2,2017-07-02,203.5,Düsseldorf,Liège,Flat stage,Marcel Kittel,GER,2017,Marcel_Kittel,None
2,3,2017-07-03,212.5,Verviers,Longwy,Medium mountain stage,Peter Sagan,SVK,2017,Peter_Sagan,Mountain stage
3,4,2017-07-04,207.5,Mondorf-les-Bains,Vittel,Flat stage,Arnaud Démare,FRA,2017,Arnaud_Démare,None
4,5,2017-07-05,160.5,Vittel,La Planche des Belles Filles,Medium mountain stage,Fabio Aru,ITA,2017,Fabio_Aru,Mountain stage


In [9]:
riders.head(3)

,rider_name,birth_date,birth_city
0,Geraint_Thomas,1986-05-25,Cardiff
1,Marcel_Kittel,1988-05-11,Arnstadt
2,Peter_Sagan,1990-01-26,Žilina


In [10]:
cities = cities.drop_duplicates()
cities.head(3)

,city,lat,lng
0,Düsseldorf,51.225402,6.776314
1,Verviers,50.593240,5.867828
2,Mondorf-les-Bains,49.507175,6.281424


In [11]:
stages = stages.join(riders.set_index('rider_name'), on='rider_name')

In [12]:
france = stages.loc[(stages['Winner_Country']=='FRA')]

In [35]:
dist_from_origin = []
dist_from_destination = []

for i in range(len(stages)):
    origin = stages['Origin'].iloc[i]
    destination = stages['Destination'].iloc[i]
    birth_city = stages['birth_city'].iloc[i]
          
    if origin in cities.city.unique():
        coords_o = (cities[cities.city == origin]['lat'].item(), cities[cities.city == origin]['lng'].item())
        if np.isnan(coords_o[0]):
            coords_o = None
    else:
        coords_o = None
        
    if destination in cities.city.unique():
        coords_d = (cities[cities.city == destination]['lat'].item(), cities[cities.city == destination]['lng'].item())
        if np.isnan(coords_d[0]):
            coords_d = None
    else:
        coords_d = None
        
    if birth_city in cities.city.unique():
        coords_b = (cities[cities.city == birth_city]['lat'].item(), cities[cities.city == birth_city]['lng'].item())
        if np.isnan(coords_b[0]):
            coords_b = None
    else:
        coords_b = None
        
        
    if (type(coords_b) == tuple) | (type(coords_o) == tuple):
        print(coords_o[0])
        dist_o = geodesic(coords_b, coords_o).kilometers
    else:
        dist_o = None
    if (type(coords_b) == tuple) | (type(coords_d) == tuple):
        dist_d = geodesic(coords_b, coords_d).kilometers
    else:
        dist_d = None
        
    dist_from_origin.append(dist_o)
    dist_from_destination.append(dist_d)

stages['dist_from_origin'] = dist_from_origin
stages['dist_from_destination'] = dist_from_destination

51.2254018
51.2254018
50.59324
49.5071747
48.2020717
47.6197395
48.2971626
47.0921047
46.1534255
45.1909365
44.6681229
43.2957547
42.9841865
43.6434
44.3691776
45.0459739
44.9038429
44.8984037
44.5641764
43.2961743
48.7038873
48.6355232
49.1157004
48.8376284
47.2596224
45.8354243
44.9034428
43.6144444
43.2957547
42.7581694
42.4869128
43.2130358
43.6112422
44.3707629
44.5579391
46.2051192
46.431063
46.9482713
45.9361246
45.6754622
45.8567089
49.1931736
52.0809856
52.0809856
51.2211097
50.6152381
50.291048
50.1060835
49.0053166
48.1113387
47.6586772
43.232858
43.2957547
43.1252298
43.4599858
44.3513062
44.5180226
45.0354375
44.0918144
44.5612032
45.2775258
45.2015369
48.8247407
53.7974185
53.9590555
52.2055314
50.5211202
50.8516968
50.291048
49.042561
48.6864409
48.0705772
47.7467233
47.2380222
46.2051192
45.4401467
45.1875602
44.4634515
43.2130358
43.1077682
43.2957547


TypeError: 'NoneType' object is not subscriptable

In [16]:
france.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 691 entries, 3 to 2235
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Stage                  691 non-null    object        
 1   Date                   691 non-null    datetime64[ns]
 2   Distance               691 non-null    float64       
 3   Origin                 691 non-null    object        
 4   Destination            691 non-null    object        
 5   Type                   691 non-null    object        
 6   Winner                 691 non-null    object        
 7   Winner_Country         691 non-null    object        
 8   Year                   691 non-null    int64         
 9   rider_name             691 non-null    object        
 10  Gen_type               652 non-null    object        
 11  birth_date             670 non-null    object        
 12  birth_city             671 non-null    object        
 13  dist

In [17]:
france.to_csv('france.csv', index=False)
stages.to_csv('stages.csv', index=False)